In [91]:
import itertools
import json
import numpy as np

from collections import Counter

In [64]:
class MCModel:
    def __init__(self) -> None:
        self.dmatrix = np.array(
            [
                [0, 0, 0],
                [0, 1, 1],
                [0, 1, 2],
            ]
        )


In [59]:
with open("../data_pcg_ready/40x23_fg/lvls_fg.json", "r") as f:
    d_levels = json.load(f)

In [79]:
test = np.array([
    ['a', 'b', 'c'],
    ['d', 'e', 'f'],
    ['g', 'h', 'i']
    ])

In [84]:
test

array([['a', 'b', 'c'],
       ['d', 'e', 'f'],
       ['g', 'h', 'i']], dtype='<U1')

In [87]:
def submatrix_to_count(array):
    return "".join((array[1, 1], array[1, 2], array[2, 1])), array[2, 2]

def get_all_submatrices(array, xmax=23, ymax=40, size=2):
    all_submatrices = []
    for x in range(xmax-size):
        for y in range(ymax-size):
            all_submatrices.append(array[x:x+3, y:y+3])
    return all_submatrices

In [86]:
submatrix_to_count(test)

('efh', 'i')

In [99]:
all_tiles = []
for lv in d_levels.values():
    all_tiles.extend(np.array(lv).flatten())
all_tiles = set(all_tiles)

d_absolute_counts = {}
for perm in itertools.product(all_tiles, repeat=3):
    d_absolute_counts["".join(perm)] = []

for lvl in d_levels.values():
    for submatrix in get_all_submatrices(np.array(lvl)):
        pattern, tiletype = submatrix_to_count(submatrix)
        d_absolute_counts[pattern].append(tiletype)

for key in d_absolute_counts.keys():
    d_absolute_counts[key] = dict(Counter(d_absolute_counts[key]))

In [98]:
l = ['b', 'b', 'b', 'b', 'f', 'f', 'b']
dict(Counter([]))

{}

In [100]:
d_absolute_counts

{'bbb': {'b': 460, '0': 20, 'f': 3},
 'bbh': {},
 'bbd': {},
 'bbf': {'b': 5, 'f': 5},
 'bb9': {},
 'bb0': {'0': 39, 'b': 18},
 'bbe': {},
 'bba': {},
 'bb6': {},
 'bb8': {},
 'bb1': {},
 'bbn': {},
 'bb3': {},
 'bb5': {},
 'bb7': {},
 'bb4': {},
 'bbm': {},
 'bbg': {},
 'bbk': {},
 'bbc': {},
 'bhb': {},
 'bhh': {},
 'bhd': {},
 'bhf': {},
 'bh9': {},
 'bh0': {},
 'bhe': {},
 'bha': {},
 'bh6': {},
 'bh8': {},
 'bh1': {},
 'bhn': {},
 'bh3': {},
 'bh5': {},
 'bh7': {},
 'bh4': {},
 'bhm': {},
 'bhg': {},
 'bhk': {},
 'bhc': {},
 'bdb': {},
 'bdh': {},
 'bdd': {},
 'bdf': {},
 'bd9': {},
 'bd0': {},
 'bde': {},
 'bda': {},
 'bd6': {},
 'bd8': {},
 'bd1': {},
 'bdn': {},
 'bd3': {},
 'bd5': {},
 'bd7': {},
 'bd4': {},
 'bdm': {},
 'bdg': {},
 'bdk': {},
 'bdc': {},
 'bfb': {'b': 5, 'f': 2},
 'bfh': {},
 'bfd': {},
 'bff': {},
 'bf9': {},
 'bf0': {},
 'bfe': {},
 'bfa': {},
 'bf6': {},
 'bf8': {},
 'bf1': {},
 'bfn': {},
 'bf3': {},
 'bf5': {},
 'bf7': {},
 'bf4': {},
 'bfm': {},
 'bfg':

In [116]:
d_proba_estimation = {}

for key in d_absolute_counts.keys():
    d_temp = d_absolute_counts[key]
    if d_temp:        
        d_proba_estimation[key] = {k: (v / total) for total in (sum(d_temp.values()),) for k, v in d_temp.items()}
        

In [117]:
d_proba_estimation

{'bbb': {'b': 0.9523809523809523,
  '0': 0.041407867494824016,
  'f': 0.006211180124223602},
 'bbf': {'b': 0.5, 'f': 0.5},
 'bb0': {'0': 0.6842105263157895, 'b': 0.3157894736842105},
 'bfb': {'b': 0.7142857142857143, 'f': 0.2857142857142857},
 'b0b': {'0': 0.9903846153846154, 'b': 0.009615384615384616},
 'b0f': {'f': 0.8333333333333334, '0': 0.16666666666666666},
 'b00': {'0': 1.0},
 'hhh': {'h': 0.9793510324483776,
  'g': 0.017699115044247787,
  '0': 0.0029498525073746312},
 'hh0': {'h': 0.5454545454545454,
  '0': 0.36363636363636365,
  'g': 0.09090909090909091},
 'hhg': {'g': 0.8611111111111112, 'h': 0.125, '0': 0.013888888888888888},
 'h0h': {'0': 0.8333333333333334,
  'h': 0.07575757575757576,
  'g': 0.09090909090909091},
 'h00': {'0': 1.0},
 'h0g': {'0': 1.0},
 'hgh': {'g': 0.8947368421052632,
  'h': 0.07894736842105263,
  '0': 0.02631578947368421},
 'hg0': {'g': 1.0},
 'hgg': {'g': 1.0},
 'ddd': {'d': 0.9772433410912852, '0': 0.022756658908714766},
 'dd0': {'0': 0.8734375, 'd': 0